In [39]:
import torch

from etnn import TreeNode
from etnn.nn.baseline import calc_params, create_baseline_model
from etnn.nn.layer_framework import LayerManagementFramework
from etnn.tools.training_tools import ConfigStore

In [40]:
config = ConfigStore(
    in_dim=15,
    hidden_dim=128,
    out_dim=1,
    k=3,
    dataset=0,
    ds_size=10_000,
    num_gondolas=10,
    num_part_pg=5,
    loss_name='mse',
    optimizer_name='adam',
    num_max_epochs=30, # real: 100
    learning_rate=0.001,
    batch_size=1024,
    early_stop_tol=5,
    use_equal_batcher=False,
    seed=420,
    label_type="tree_advanced",
    final_label_factor=1/1000
)

In [41]:
tree_structure = TreeNode(
    node_type="C",
    children=[
        TreeNode("P", [TreeNode("E", config.num_part_pg)])
        for _ in range(config.num_gondolas)
    ]
)

In [42]:
model_tree = LayerManagementFramework(
    in_dim=config.in_dim,
    tree=tree_structure,
    hidden_dim=config.hidden_dim,
    out_dim=config.out_dim,
    k=config.k
)

In [43]:
model_baseline, _ = create_baseline_model(
    n_params=calc_params(model_tree),
    input_dim=config.in_dim * config.num_gondolas * config.num_part_pg,
    n_layer=3,
    output_dim=1
)

In [44]:
calc_params(model_tree)

111489

In [45]:
calc_params(model_baseline)

116764

# Second part with entry csv for baseline

In [46]:
import pandas as pd

In [47]:
label = "tree_advanced"
i = 0

In [48]:
df = pd.read_csv(f"./finalTests/study_label-tree-advanced_baseline_normalized.csv")

In [49]:
df

,Unnamed: 0,number,value,datetime_start,datetime_complete,duration,params_batcher,params_learning_rate,params_n_dim_0,params_n_dim_1,params_n_dim_2,params_n_dim_3,params_n_layers,state
0,0,0,-939.444901,2023-12-08 19:56:49.744254,2023-12-08 19:57:58.943975,0 days 00:01:09.199721,False,0.000024,NaN,NaN,NaN,NaN,1,COMPLETE
1,1,1,0.920114,2023-12-08 19:57:58.945480,2023-12-08 19:59:19.023901,0 days 00:01:20.078421,False,0.002104,219.0,401.0,355.0,NaN,4,COMPLETE
2,2,2,-1004.729063,2023-12-08 19:59:19.025902,2023-12-08 20:00:40.961248,0 days 00:01:21.935346,False,0.000031,89.0,304.0,315.0,1.0,5,COMPLETE
3,3,3,0.896207,2023-12-08 20:00:40.963248,2023-12-08 20:02:05.507228,0 days 00:01:24.543980,False,0.000293,208.0,NaN,NaN,NaN,2,COMPLETE
4,4,4,0.905588,2023-12-08 20:02:05.509229,2023-12-08 20:03:23.213526,0 days 00:01:17.704297,True,0.000728,250.0,NaN,NaN,NaN,2,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,65,65,0.939706,2023-12-08 21:14:01.704534,2023-12-08 21:15:14.487481,0 days 00:01:12.782947,True,0.003787,360.0,153.0,NaN,NaN,3,COMPLETE
66,66,66,0.944978,2023-12-08 21:15:14.488480,2023-12-08 21:16:23.830428,0 days 00:01:09.341948,True,0.005396,446.0,56.0,NaN,NaN,3,COMPLETE
67,67,67,0.931577,2023-12-08 21:16:23.832429,2023-12-08 21:17:36.604296,0 days 00:01:12.771867,True,0.007961,485.0,19.0,NaN,NaN,3,COMPLETE
68,68,68,0.923358,2023-12-08 21:17:36.605310,2023-12-08 21:18:54.544129,0 days 00:01:17.938819,True,0.005931,387.0,262.0,NaN,NaN,3,COMPLETE


In [50]:
entry = df.iloc[0]

In [51]:
layer_list = [torch.nn.Flatten()]
features = config.in_dim * config.num_gondolas * config.num_part_pg

# for each layer create a linear layer and relu (except last one)
for layer in range(int(entry.params_n_layers)-1):
    # determine new feature dimension
    new_features = int(entry[f"params_n_dim_{layer}"])

    # add layer and relu to list
    layer_list += [torch.nn.Linear(features, new_features), torch.nn.ReLU()]

    # set the new feature to be the current feature
    features = new_features

# set the last layer - this one must map to the out dimension
layer_list += [torch.nn.Linear(features, config.out_dim)]
model = torch.nn.Sequential(*layer_list)

In [53]:
calc_params(model)

751